In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS f1_presentation.calculated_race_results
(
race_year INT,
team_name STRING,
driver_id INT,
driver_name STRING,
race_id INT,
position INT,
points INT,
calculated_points INT,
created_date TIMESTAMP,
updated_date TIMESTAMP
)
USING DELTA
""")

Out[25]: DataFrame[]

In [0]:
spark.sql(f""" 
CREATE OR REPLACE TEMP VIEW race_result_updated
AS
select ra.race_year, 
       c.name as team_name, 
       d.driver_id as driver_id,
       d.name as driver_name,
       ra.race_id,
       r.position,
       r.points,
       11 - r.position as calculated_points
 from f1_processed.results r join
      f1_processed.drivers d on (r.driver_id = d.driver_id) join
      f1_processed.constructors c on (r.constructor_id = c.constructor_id) join
      f1_processed.races ra on (r.race_id = ra.race_id)
where r.position <= 10 
  and r.file_date = '{v_file_date}'
""")

Out[26]: DataFrame[]

In [0]:
spark.sql(f"""
MERGE INTO f1_presentation.calculated_race_results target
USING race_result_updated source
ON (target.driver_id = source.driver_id and target.race_id = source.race_id)
WHEN MATCHED THEN
  UPDATE SET
    target.position = source.position,
    target.points = source.points,
    target.calculated_points = source.calculated_points,
    target.updated_date = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (race_year, team_name, driver_id, driver_name, race_id, position, points, calculated_points, created_date)
       VALUES (source.race_year, source.team_name, source.driver_id, source.driver_name, source.race_id, source.position, source.points, source.calculated_points, current_timestamp)
""")

Out[27]: DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
--select * from f1_presentation.calculated_race_results where race_year = 2021

race_year,team_name,driver_id,driver_name,race_id,position,points,calculated_points,created_date,updated_date
2021,Mercedes,1,Lewis Hamilton,1053,2,19,9,2022-04-21T21:35:30.037+0000,null
2021,Alpine F1 Team,4,Fernando Alonso,1053,10,1,1,2022-04-21T21:35:30.037+0000,null
2021,AlphaTauri,842,Pierre Gasly,1053,7,6,4,2022-04-21T21:35:30.037+0000,null
2021,McLaren,817,Daniel Ricciardo,1053,6,8,5,2022-04-21T21:35:30.037+0000,null
2021,Red Bull,830,Max Verstappen,1053,1,25,10,2022-04-21T21:35:30.037+0000,null
2021,Ferrari,832,Carlos Sainz,1053,5,10,6,2022-04-21T21:35:30.037+0000,null
2021,Alpine F1 Team,839,Esteban Ocon,1053,9,2,2,2022-04-21T21:35:30.037+0000,null
2021,Aston Martin,840,Lance Stroll,1053,8,4,3,2022-04-21T21:35:30.037+0000,null
2021,Ferrari,844,Charles Leclerc,1053,4,12,7,2022-04-21T21:35:30.037+0000,null
2021,McLaren,846,Lando Norris,1053,3,15,8,2022-04-21T21:35:30.037+0000,null
